<a href="https://colab.research.google.com/github/hwon-n/section2_project/blob/main/Project_script.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Section2 Project

## 데이터 선택 이유는?

은행에서 고객에게 대출을 해줄 때 채무 불이행 가능성이 있는 사람을 예측하기 위해서 머신러닝 모델을 만들어봤습니다. 새로운 고객을 확보할 때 어느 고객이 채무 불이행 가능성이 있는지에 대해서 예측할 것입니다. 

## 해당 특성을 Target으로 설정한 이유는?

Risk_Flag는 해당 고객이 이전에도 채무 불이행을 했는가에 대한 True / False로 나눠진 특성이었기 때문에 채무 불이행 가능성이 있는 사람을 예측하기 위해서는 해당 특성을 Target으로 설정하는 것이 타당함. 


=> 타겟 설정 후 얻게 되는 기대 효과 혹은 이득에 대해서 간단하게 설명해보기.

## 데이터 전처리

1) Age

 30대 미만 / 30대/ 4-50대 / 60대 ~
 총 네 그룹으로 나눠서 OrdinalEncoder로 mapping 해주기

 2) STATE

 STATE.unique()로 값들을 살펴보니 인도의 지역명이었고, 대부분의 지역들이 북부 쪽에 위치해있었다. 때문에 북부에 위치한 지역이 경우 1을 아닌 경우는 0을 return 해서 TargetEncoder로 인코딩 하기로 결정했다. 


 3) Experience

 Experience가 2년 이하면 Entry, 2 ~ 5년 사이면 Intermediate, 5 ~ 10년 사이면 Mid 그리고 그 이상이면 Senior로 그룹을 나눠줬다. 그리고 후에 Entry에 1, Intermediate에 2, Mid에 3, Senior에 4를 mapping 해줬다.

 4) ID, CITY
 ID는 모델을 학습하는데에 있어서 필요없는 값이라 제거해줬다. 또한 CITY는 너무 많은 범주를 가지고 있어서 모델이 학습하는데 일반화가 잘 안될것 같아서 제거해줬다. 

 5) Income
 우선 MinMaxScaler로 0.0 ~ 1.0 사이로 스케일링 해준 뒤 label을 사용해 pd.cut으로 라벨링을 해 준뒤 OrdinalEncoder로 Low, Mid, High로 인코딩 해줬다. 

 => 시간 제한때문에 데이터에 대한 설명을 최대한 간략하게 하다보니 데이터셋에 대해서 조금 더 설명해줬으면 좋겠다는 내용의 피드백을 받았음. 데이터셋에 대한 내용과 전처리를 어떻게 진행했는지에 대해 쉽게 설명할 방법 찾아보기. 

## Data Leakage

Data Leakage를 방지하기 위해 pipeline을 사용했음. pipeline을 사용했기 때문에 train, val, test 데이터의 손실을 피할 수 있었음. 

## 모델 학습

### 1) Logistic Regression

Training score: 0.8769

Validation score: 0.8770

MAE: 0.12299

R2: -0.140

원래 R2 값은 0과 1 사이의 값을 가지는데 음수가 나왔으므로 해당 모델은 성능이 떨어지는 것을 의미함.

### 2) Decision Tree

Validation score: 0.88075

ROC AUC score: 0.698741

f1 score: 0.48544

precision/recall: (0 - 0.93 / 0.94), (1 - 0.52 / 0.46)

### 3) RandomForest (Tree Ensemble)
Validation score: 0.088789

ROC AUC score: 0.70359

f1 score: 0.50187

precision/recall : (0 - 0.93/0.95) , (1 - 0.55/0.46)

## 최종 모델 결정

Logistic Regression의 R2 score를 보면 음수가 나왔으므로 이 데이터에 Logistic Regression을 사용하는 것은 적합하지 않다는 판단이 들어 Logistic Regression을 제외했습니다.

또한, Decision Tree와 RandomForest를 비교했을 때, 둘의 Validation score가 비슷하지만 Randomforest쪽이 ROC AUC score와 f1 score가 더 높으므로 Decision tree보다 Randomforest 모델이 제가 사용하려는 데이터에 적합하다는 결론이 내려져 최종 모델은 "RandomForest"로 결정했습니다. 

## 하이퍼파라미터 튜닝

Decision Tree나 Randomforest를 보면 모두 과적합이므로 최대한 과적합을 제어해주기 위해서 트리의 노드를 정하는 최대 특성의 수를 qrt 해줬고, 리프 노드의 최소 수를 5개로 제한해주었다. (최소 수가 커질수록 과적합 위험이 높아짐)

또한 target이 불균형 클래스이므로 class_weight을 balanced로 제한해줬다. 
기존에는 총 100개의 모델을 만들어서 최종적으로 결론을 냈지만, 200개의 모델로 늘려서 다양한 결과를 도출하고 이를 바탕으로 결론을 낼 수 있도록 했다. 


그 결과 validation 검증 정확도는 0.8576으로 낮아졌지만 ROC AUC score는 0.83629로, f1 score는 0.58262로 올랐다.

그래서 test dataset에도 최종적으로 검증을 해봤는데, test 검증 정확도는 0.85670이고 ROC AUC score는 0.83023, f1 score는 0.57716으로 나쁘지 않은 성적을 보여줬다. 


=> RandomSearchCV를 못돌렸는데 데이터의 크기를 줄여서 시도해보고, 다시 한번 하이퍼파라미터 튜닝을 해보기. 

## 모델 시각화

### PermutationImportance

모델의 성능을 저하시키는 특성이 있으면 제외하려고 Permutation Importance를 시행했는데 다행스럽게도 낮은 score는 있었지만 음수를 가진 score는 없었기 때문에 추가적인 feature engineering은 하지 않아도 됐다.

=> Permutation Importance 외에도 PDP 라이브러리를 사용해서 컬럼간 연관성이 있는지도 한번 살펴보기 (ex. 연봉과 직업의 관련도, 연봉과 현재 직장 근무 년수 등의 컬럼끼리의 관련을 살펴보고 설명해보기)


### SHAP

빨간색은 예측에 긍정적인 영향을, 파란색은 예측에 부정적인 영향을 줌. 

=> 해당 내용을 영상에서 빨간 부분은 채무 불이행에 대해 긍정적인 영향을 끼쳤다고 설명했는데 이해하기 힘들다는 피드백을 받았으므로 다르게, 더 쉽게 설명할 수 있게 확실하게 개념 잡고 가기. 

=> 결론을 내리면서 채무 불이행을 한 사람들의 경우 어떠한 특성을 갖고 있다는 점을 분석하면 좋을 것 같다는 피드백을 받았으므로 한번 해보기! 